In [2]:
import pandas as pd
import json
import folium
import numpy as np
import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm

import time

from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

/tmp/ipykernel_22024/1392742312.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
ediya_driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
ediya_driver.maximize_window()
ediya_driver.get("https://www.ediya.com/contents/find_store.html#c")


In [3]:
gu_list = [
    '강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구',
    '노원구','도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구',
    '성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구'
    ]

In [4]:
ediya_stores = []

for gu  in tqdm(gu_list) :

    ediya_driver.find_element(By.XPATH,"""//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a""").click()
    time.sleep(1)
    ediya_driver.find_element(By.ID,"keyword").send_keys("서울 " + gu + Keys.ENTER)
    time.sleep(1) 

    html = ediya_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    ediya_store = soup.find_all("li","item")

    for each in ediya_store :
        ediya_store_info = [
            gu, #re.search("\s\w+[ \S]*", each.text).group().split()[1],
            re.search("\w+ ", each.text).group()[:-1],
            re.search("\s\w+[ \S]*", each.text).group()[1:].split("(")[0],
            ''
        ]
        
        ediya_stores.append(ediya_store_info)

  0%|          | 0/25 [00:00<?, ?it/s]

In [5]:
for i in tqdm(range(len(ediya_stores))):
    lat_lng = geocoding(ediya_stores[i][2])
    ediya_stores[i][2] = lat_lng[0]
    ediya_stores[i][3] = lat_lng[1]

  0%|          | 0/724 [00:00<?, ?it/s]

In [15]:
ediya = pd.DataFrame(ediya_stores)
ediya.head()

,0,1,2,3
0,강남구,강남YMCA점,37.512302,127.029494
1,강남구,강남구청역아이티웨딩점,37.519970,127.056947
2,강남구,강남논현학동점,37.514393,127.029435
3,강남구,강남대치점,37.504118,127.059377
4,강남구,강남도산점,37.522084,127.031342


In [1]:
mapping = folium.Map(location=[37.53, 127], width="100%", height="100%", zoom_start=11.8)
    
for idx, row in ediya.iterrows():
    folium.Circle(
        location=[row[2], row[3]],
        radius=100,
        color='blue',
        fill=True,
        fill_color='blue'
        ).add_to(mapping)

mapping

NameError: name 'folium' is not defined